# Code for Calculating the Frobenius Norm of target unitary from different GRAPE functions

In this notebook, we use two different functions from the qutip control library ("optimize_pulse_unitary" from the pulseoptim object and "cy_grape_unitary" from the grape object), in order to implement the GRAPE algorithm. From this we obtain a final Unitary $\hat U$ which should be close to some given target Unitary $\hat U^*$. We then calculate the Frobenius norm squared of the difference between these two unitary as a measure of the error fidelity:

$$ || A  ||_{F}^2  = Tr(A^{\dagger}A) = \sum_{ij}^{N} |A_{ij}|^{2}$$


These values are then plotted against previously obtained results which obtained the unitary by reformulating the system in terms of a polynomial equation (https://arxiv.org/pdf/2209.05790.pdf) 

## Imports

Below are the necessary imports for running this code:

1) Qutip functions: 


The first line does import all qutip functions from the base object, which is used here to convery numpy arrays to Quantum objects using Qobj() and qeye() to create an identity operator of a specified size (here 3) as the initial starting point for the GRAPE algorithm.

The next set of imports are functions used for the pulseoptim class and a logger class which formats the result when being printed to terminal/ saving the to a file (I don't think we need them)

The final set of qutip imports are for implementing the cy_grape method. The TextProgressBar is justed used as a convenient way of tracking the progress of the function. (plot_grape_control_fields and _overlap are not necessary any more but are just used as tools for plotting the grape control fields and calculating the trace norm respectively)

2) Other imports:


The next set of imports are just the basic imports for simple tasks: 
matplotlib for plotting. 

numpy for storing and manipulating data.

h5py for reading and writing hdf5 files. 

time for time-keeping.  


In [1]:
from qutip import *

#QuTiP control modules
import qutip.control.pulseoptim as cpo
import qutip.logging_utils as logging
logger = logging.get_logger()
#Set this to None or logging.WARN for 'quiet' execution
log_level = logging.INFO

from qutip.control import * 
from qutip.ui.progressbar import TextProgressBar
from qutip.control.grape import plot_grape_control_fields, _overlap


%matplotlib inline
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
#import datetime
import h5py
import time 
start_time = time.time()

from tqdm.notebook import tqdm

from multiprocessing import Pool

## Preparing the input data
 
 

 
In the cell below we read data from hdf5 files. The first is a list of 1000 3x3 target unitaries and the second is the frobenius norm calculated within the aforementioned paper. 

*Note: Bottom half of this cell may change as appending to an array can be computationally expensive* 

Sample here is used to denote the number of unitaries from the list that we would like to select and a for-loop is used to convert these unitaries into Qobj type which are stored in an array. Two empyt lists are also initialised which are used for storing data. 


In [2]:
#Read in the hdf5 files

with h5py.File("../results.hdf5", 'r') as resultsFile:
    #extract the data and store in an array

    U_targets = resultsFile["U_targets"][...]
    norm_U_target_minus_obtainedFile = resultsFile["norm_U_target_minus_obtained"][...]
    f_PSU = resultsFile["f_PSU"][...]
    
sample = U_targets.shape[2] #choose number of target unitaries - this is choosing the full 1000

#for storing the obtained unitaries (*)
U_final_pulseoptim = []
U_final_cyGRAPE = []

## Choosing input parameters

The dynamics of the system are generated by the time dependent Hamiltionian:

$$H(t) = H_{0} + \sum_{i} u_i(t)H_{i} $$

Where $H_{0}$ is the drift Hamiltonian, $H_{i}$ are the control Hamiltonians and $u_{i}$ are the control fields. 

In the cell below we create Qobjs for $H_{0}$ and the single control $H_{c}$ according to the previously mentioned paper. We choose the Identity operator as the starting point for the GRAPE algorithm. We also can set the period over which the algorithm runs and the number of time slots at which act as the number of steps taken find a minimum in the landscape. 

(See https://qutip.org/docs/latest/guide/guide-control.html and the paper above for a more detail discussion).

In [3]:
H_drift_matrix = np.array(
    [[0, 0, 0],
     [0, 3.21505101e+10, 0],
     [0, 0, 6.23173079e+10]]
)
H_drift_matrix /= H_drift_matrix.max()

H_control_matrix = np.array(
    [[0, 1, 0],
     [1, 0, 1.41421356],
     [0, 1.41421356, 0]]
)
H_control_matrix /= H_control_matrix.max()

H_drift = Qobj(H_drift_matrix)
H_control = [Qobj(H_control_matrix)] 
 
# Unitary starting point
U_0 = qeye(3)


# Number of time slots
n_ts = 10

# Time allowed for the evolution
evo_time = 0.5

## pulseoptim

In the two cells below we run the function from the pulseoptim subclass. The first cell we select some more parameters for this function, these are:
 1) fid_error_tar - when this fidelity error is achieved we can terminate the algorithm as we have sufficiently achieved our goal.
 2) max_iterations - the max number of grape iterations 
 3) max_wall_time - maximum time the algorithm runs before terminating if the target unitary has not been reached
 4) min_grad - minimum gradient to determine whether we are trapped in a local minima rather than a global minima 
 5) p_type - the type of pulse for the control fields (here set to random)
 
 
 The second cell then runs the algorithm with these parameters for each unitary in the sample size. The finial unitary is stored in the "U_final_pulseoptim" list and for each iteration the termination reason and the fidelity error is printed out. You can also extract other useful information from the result variable such as the final control fields, the number of iterations used etc. 


In [4]:
#pulse optim extra params 

# Fidelity error target
fid_err_targ = 1e-10
# Maximum iterations for the optisation algorithm
max_iter = 1000
# Maximum (elapsed) time allowed in seconds
max_wall_time = 1000
# Minimum gradient (sum of gradients squared)
# as this tends to 0 -> local minima has been found
min_grad = 1e-10

# pulse type alternatives: RND|ZERO|LIN|SINE|SQUARE|SAW|TRIANGLE|
p_type = 'SINE' 

In [5]:
def optimize(U_tag, alg):
    return cpo.optimize_pulse_unitary(
                H_drift, H_control, U_0, Qobj(U_tag), n_ts, evo_time, 
                fid_err_targ=fid_err_targ, min_grad=min_grad, 
                max_iter=max_iter, max_wall_time=max_wall_time, 
                # log_level=log_level,
                init_pulse_type=p_type, 
                alg=alg,
                # gen_stats=True
            ).evo_full_final

In [6]:
U_final_pulseoptim = [
    optimize(U_targets[:,:,i], alg='CRAB') for i in tqdm(range(sample))
]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
U_final_cyGRAPE = [
    optimize(U_targets[:,:,i], alg='GRAPE') for i in tqdm(range(sample))
]

  0%|          | 0/1000 [00:00<?, ?it/s]


for i in tqdm(range(sample)):
    i = 300
    result = cpo.optimize_pulse_unitary(
                H_drift, H_control, U_0, Qobj(U_targets[:,:,i]), n_ts, evo_time, 
                fid_err_targ=fid_err_targ, min_grad=min_grad, 
                max_iter=max_iter, max_wall_time=max_wall_time, 
                # log_level=log_level,
                init_pulse_type=p_type, 
                #alg='GRAPE',
                alg='CRAB',
                gen_stats=False
            )
    print("Final fidelity error {}".format(result.fid_err))
    print("Terminated due to {}".format(result.termination_reason))
   
    #Here all terminate due to convergence however if it terminates due to a different reason may cause a problem
    #U_final_pulseoptim.append(result.evo_full_final)
    
    break

## cy_grape 

In these two cells below we set the the parameters and run the cy_grape_unitary function. There are less inputs in this method:

1) times - an array of where the number of elements = number of time slots and the final element is the period.
2) n_iterations - number of grape iterations

the second cell follows the same procedure as the cell above, however in this case we can track the progress of each iteration with TextProgressBar(). Here we only the store the final unitary, however one could extract other information from the result variable such as the final control fields (for each grape iteration). 


## Plotting Fidelity error (trace norm)
This code is for calculating the trace norm (overlap) of the obtained and target unitaries:

$\frac{1}{d} |Tr[\hat U^{\dagger}\hat U^{*}]| $,

$d$ is the dimension of $\hat U$.

(currently not plotting this just calculating it. The cells for plotting have been commented out and folded as they still use TSSOS but will modify if needed)




In [10]:
#calculating fidelity

pulseoptim_fidelity = np.zeros(sample)
cy_grape_fidelity = np.zeros(sample)


for i in range(sample):
    pulseoptim_fidelity[i] = abs(_overlap(Qobj(U_targets[:,:,i]), U_final_pulseoptim[i]))
    cy_grape_fidelity[i] = abs(_overlap(Qobj(U_targets[:,:,i]), U_final_cyGRAPE[i]))
    


pulseoptim_fid_error = 1 - pulseoptim_fidelity 
cy_grape_fid_error = 1 - cy_grape_fidelity


# Writing data to hdf5

In [9]:
with h5py.File('pulseoptim_fid_error_timeslices_' +str(n_ts) + '.hdf5', 'w') as hf:
    hf.create_dataset("pulseoptim_fid_error_timeslices_" +str(n_ts),  data=pulseoptim_fid_error)
    
with h5py.File('cy_grape_fid_error_timeslices_' + str(n_ts) + '.hdf5', 'w') as hf:
     hf.create_dataset("cy_grape_fid_error_timeslices_" + str(n_ts),  data=cy_grape_fid_error)